In [ ]:
from abc import ABC, abstractmethod

class PaymentStrategy(ABC):
    @abstractmethod
    def process_payment(self, amount):
        pass


In [ ]:
class PaymentDecorator(PaymentStrategy):
    def __init__(self, payment_strategy: PaymentStrategy):
        self._wrapped_strategy = payment_strategy

    @abstractmethod
    def process_payment(self, amount):
        pass

class DiscountDecorator(PaymentDecorator):
    def __init__(self, payment_strategy, discount_rate):
        super().__init__(payment_strategy)
        self.discount_rate = discount_rate  # e.g., 0.10 for 10% discount

    def process_payment(self, amount):
        discounted_amount = amount * (1 - self.discount_rate)
        print(f"Applying a discount of {self.discount_rate * 100:.0f}%")
        return self._wrapped_strategy.process_payment(discounted_amount)
      
class FeeDecorator(PaymentDecorator):
    def __init__(self, payment_strategy, fee_rate):
        super().__init__(payment_strategy)
        self.fee_rate = fee_rate  # e.g., 0.02 for 2% fee

    def process_payment(self, amount):
        fee = amount * self.fee_rate
        amount_with_fee = amount + fee
        print(f"Applying a fee of {self.fee_rate * 100:.0f}% (${fee:.2f})")
        return self._wrapped_strategy.process_payment(amount_with_fee)

In [9]:
class CreditCardPayment(PaymentStrategy):
    def process_payment(self, amount):
        print(f"Processing credit card payment of ${amount:.2f}")
        return amount

class PayPalPayment(PaymentStrategy):
    def process_payment(self, amount):
        print(f"Processing PayPal payment of ${amount:.2f}")
        return amount

class CryptoPayment(PaymentStrategy):
    def process_payment(self, amount):
        print(f"Processing cryptocurrency payment of ${amount:.2f}")
        return amount

In [ ]:
class PaymentContext:
    def __init__(self, payment_strategy: PaymentStrategy = None):
        self.payment_strategy = payment_strategy

    def set_payment_strategy(self, payment_strategy):
        self.payment_strategy = payment_strategy

    def process_payment(self, amount):
        if not self.payment_strategy:
            print("No payment strategy set.")
            return
        return self.payment_strategy.process_payment(amount)


In [12]:
def main():
    # Original payment strategies
    credit_card_payment = CreditCardPayment()
    paypal_payment = PayPalPayment()
    crypto_payment = CryptoPayment()

    # User selects payment method
    payment_methods = {
        "1": ("Credit Card", credit_card_payment),
        "2": ("PayPal", paypal_payment),
        "3": ("Cryptocurrency", crypto_payment)
    }

    print("\nSelect a payment method:")
    for key, (name, _) in payment_methods.items():
        print(f"{key}. {name}")

    choice = input("Enter the number of your choice: ")

    if choice not in payment_methods:
        print("Invalid choice")
        return

    selected_name, selected_strategy = payment_methods[choice]

    amount = float(input("Enter the amount to be paid: $"))

    # Ask if user wants to apply a discount
    apply_discount = input("Apply discount? (y/n): ").lower() == 'y'
    if apply_discount:
        discount_rate = float(input("Enter discount rate (e.g., 0.10 for 10%): "))
        selected_strategy = DiscountDecorator(selected_strategy, discount_rate)

    # Ask if user wants to apply a fee
    apply_fee = input("Apply transaction fee? (y/n): ").lower() == 'y'
    if apply_fee:
        fee_rate = float(input("Enter fee rate (e.g., 0.02 for 2%): "))
        selected_strategy = FeeDecorator(selected_strategy, fee_rate)

    # Process payment
    total_paid = selected_strategy.process_payment(amount)
    print(f"Initial Price: ${amount:.2f}")
    print(f"Payment Method: {selected_name}")
    print(f"Total amount charged: ${total_paid:.2f}")

if __name__ == "__main__":
    main()



Select a payment method:
1. Credit Card
2. PayPal
3. Cryptocurrency
Applying a fee of 2% ($18000.00)
Applying a discount of 20%
Processing credit card payment of $734400.00
Initial Price: $900000.00
Payment Method: Credit Card
Total amount charged: $734400.00
